In [10]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
blind = pd.read_csv("../Data/Data_submission/test_blind.csv", sep=";") #fichier csv qui sera merge pas les prédiction et soumis sur Kaggle
donnees_abiotiques = pd.read_csv("../Data/Data_train/enviroTab_pa_train.csv", sep = ';')  #fichier csv données abiotiques des patchID
presences = pd.read_excel("../Data/Data_train/Presences_Absences_train.xlsx") #fichier excel presences/absences

In [32]:
donnees_abiotiques

,patchID,dayOfYear,lon,lat,x_EPSG3035,y_EPSG3035,year,dataset,geoUncertaintyInM,bio1,...,Built1994,Lights1994,NavWater1994,Built2009,Lights2009,NavWater2009,Popdensity1990,Popdensity2010,Railways,Roads
0,55880,126,4.74408,44.20370,3.900369e+06,2.358486e+06,2020,CBNMed,10.0,2878,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
1,55897,126,4.74436,44.20410,3.900395e+06,2.358529e+06,2020,CBNMed,10.0,2878,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
2,55942,126,4.74519,44.20360,3.900457e+06,2.358469e+06,2020,CBNMed,10.0,2878,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
3,56533,126,4.75466,44.21930,3.901329e+06,2.360153e+06,2020,CBNMed,10.0,2876,...,0.0,4.0,0.144801,0.0,5.0,0.144801,6.0,6.0,0.0,0.852498
4,56759,126,4.75755,44.21360,3.901518e+06,2.359506e+06,2020,CBNMed,10.0,2877,...,0.0,4.0,0.144801,0.0,5.0,0.144801,6.0,6.0,0.0,0.852498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,3018845,138,9.28662,41.87388,4.261560e+06,2.085921e+06,2018,Inventaire forestier IGN,1.0,2839,...,0.0,0.0,0.002858,0.0,0.0,0.002858,2.0,2.0,0.0,0.757507
5944,3018847,351,9.29834,41.56691,4.262244e+06,2.051949e+06,2018,Inventaire forestier IGN,1.0,2898,...,0.0,3.0,1.281452,0.0,6.0,1.281452,5.0,6.0,0.0,1.629548
5945,3018848,346,9.31529,41.70986,4.263796e+06,2.067751e+06,2017,Inventaire forestier IGN,1.0,2894,...,0.0,1.0,0.042134,0.0,1.0,0.042134,4.0,4.0,0.0,4.000000
5946,3018853,346,9.35871,41.67093,4.267389e+06,2.063412e+06,2018,Inventaire forestier IGN,1.0,2897,...,0.0,3.0,1.788531,0.0,4.0,1.788531,3.0,4.0,0.0,0.526984


In [ ]:
def k_voisins(k, data, test):
    """
    données:
        k: nombre de voisins considérés 
        data: dataFrame contenant entre autre les patch_ID et les espèces associées du train-PA
        test: dataFrame provenant du CSV Kaggle contenant les informations sur les lieux où la prédiction
        doit être faite
    fonction: 
        Liste les espèces présentes dans les k-voisins les plus proches de chaque coordonnées du dataFrame
        test à partir des information de data. 
    résultat:
        resultat: Un fichier CSV contenant les prédicitions d'especes dans la colonne Predicted 
    """
    # data frame contenant les espèces_id selon la lat,lon où l'espèce est recensées (sans doublons)
    data_sans_doublons = data.groupby(['lat', 'lon'])['speciesId'].apply(set).reset_index()
    
    # Données utilisées par l'algo 
    X1 = test[['lat', 'lon']].values
    X2 = data_sans_doublons[['lat', 'lon']].values
    
    # Création du modèle
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(X2)
    # le modèle retourne l'indice des k-voisins ainsi que leurs distances du point
    distances, indices = nn.kneighbors(X1)
    
    # Récupération des espèces selon les indices des coordonnées des voisins
    result = []
    for i in range(len(test)):
        nearest_species = set()
        for j in range(len(indices[i])):
            index = indices[i][j]
            nearest_species.update(data_sans_doublons.loc[index]['speciesId'])
        result.append(list(nearest_species))
    
    #traitement pour l'export du CSV au format acceptable par Kaggle
    test['nearest_species'] = result
    test = test[['Id', 'nearest_species']]
    test = test.rename(columns={'nearest_species': 'Predicted'})    
    test['Predicted'] = test['Predicted'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace(',', ''))
    test.to_csv('30-voisins.csv', index=False, sep=',')

    return test


In [84]:
blind

,Id,datasetName,date,dayOfYear,year,patchID,timeSerieID,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM
0,1,CBNMed,1629064800,228,2021,10125,3988380,1.729607,42.52171,3.640762e+06,2.194746e+06,5.0
1,2,CBNMed,1629064800,228,2021,10126,3997090,1.729624,42.52177,3.640764e+06,2.194753e+06,4.0
2,3,CBNMed,1629064800,228,2021,10127,4001106,1.729652,42.52197,3.640769e+06,2.194775e+06,5.0
3,4,CBNMed,1629064800,228,2021,10129,4004605,1.729671,42.52192,3.640770e+06,2.194769e+06,6.0
4,5,CBNMed,1531346400,193,2018,10154,3987660,1.826847,42.57452,3.649316e+06,2.199695e+06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22399,22400,Inventaire forestier IGN,2018-03-14,73,2018,3018864,3990152,9.421020,42.38755,4.273164e+06,2.142680e+06,1.0
22400,22401,Inventaire forestier IGN,2018-03-22,81,2018,3018865,3986840,9.429860,42.26077,4.273796e+06,2.128639e+06,1.0
22401,22402,Inventaire forestier IGN,2017-03-13,72,2017,3018866,3993669,9.462680,42.92578,4.277004e+06,2.202255e+06,1.0
22402,22403,Inventaire forestier IGN,2017-03-15,74,2017,3018867,3993567,9.497790,42.41844,4.279528e+06,2.146054e+06,1.0


In [83]:
blind_abiotique = pd.read_csv("../Data/Data_submission/enviroTab_pa_test.csv", sep = ';')
blind_abiotique

,Id,datasetName,dayOfYear,year,patchID,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,...,Built1994,Lights1994,NavWater1994,Built2009,Lights2009,NavWater2009,Popdensity1990,Popdensity2010,Railways,Roads
0,1,CBNMed,228,2021,10125,1.729607,42.52171,3.640762e+06,2.194746e+06,5.0,...,0,5,0.000000,10,9,0.000000,5,6,0,1.629548
1,2,CBNMed,228,2021,10126,1.729624,42.52177,3.640764e+06,2.194753e+06,4.0,...,0,5,0.000000,10,9,0.000000,5,6,0,1.629548
2,3,CBNMed,228,2021,10127,1.729652,42.52197,3.640769e+06,2.194775e+06,5.0,...,0,5,0.000000,10,9,0.000000,5,6,0,1.629548
3,4,CBNMed,228,2021,10129,1.729671,42.52192,3.640770e+06,2.194769e+06,6.0,...,0,5,0.000000,10,9,0.000000,5,6,0,1.629548
4,5,CBNMed,193,2018,10154,1.826847,42.57452,3.649316e+06,2.199695e+06,0.0,...,0,0,0.000000,0,2,0.000000,1,2,0,2.728224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22399,22400,Inventaire forestier IGN,73,2018,3018864,9.421020,42.38755,4.273164e+06,2.142680e+06,1.0,...,0,0,0.006391,0,0,0.006391,3,3,0,0.757507
22400,22401,Inventaire forestier IGN,81,2018,3018865,9.429860,42.26077,4.273796e+06,2.128639e+06,1.0,...,0,0,0.002733,0,0,0.002733,3,3,0,0.526984
22401,22402,Inventaire forestier IGN,72,2017,3018866,9.462680,42.92578,4.277004e+06,2.202255e+06,1.0,...,0,0,4.000000,0,0,4.000000,3,4,0,8.000000
22402,22403,Inventaire forestier IGN,74,2017,3018867,9.497790,42.41844,4.279528e+06,2.146054e+06,1.0,...,0,5,0.661320,0,5,0.661320,5,6,0,1.629548


In [87]:
X1 = blind_abiotique[['bio1', 'bio2','bio3','bio4','bio5','bio6','bio7','bio8','bio9','bio1','bio10',
                      'bio11','bio12','bio13','bio14','bio15','bio16','bio17','bio18','bio19']].values
X2 = data_sans_doublons[['bio1', 'bio2','bio3','bio4','bio5','bio6','bio7','bio8','bio9','bio1','bio10',
                      'bio11','bio12','bio13','bio14','bio15','bio16','bio17','bio18','bio19']].values

array([[2807.        ,   78.        ,    3.25999999, ..., 2121.        ,
        2438.        , 2532.        ],
       [2807.        ,   78.        ,    3.25999999, ..., 2121.        ,
        2438.        , 2532.        ],
       [2807.        ,   78.        ,    3.25999999, ..., 2121.        ,
        2438.        , 2532.        ],
       ...,
       [2832.        ,   68.        ,    3.52999997, ..., 1550.        ,
        1623.        , 1890.        ],
       [2832.        ,   68.        ,    3.52999997, ..., 1550.        ,
        1623.        , 1890.        ],
       [2832.        ,   68.        ,    3.52999997, ..., 1550.        ,
        1623.        , 1890.        ]])

In [89]:
 # Création du modèle
nn = NearestNeighbors(n_neighbors=3)
nn.fit(X2)
# le modèle retourne l'indice des k-voisins ainsi que leurs distances du point
distances, indices = nn.kneighbors(X1)
# Récupération des espèces selon les indices des coordonnées des voisins
result = []
for i in range(len(blind)):
    nearest_species = set()
    for j in range(len(indices[i])):
        index = indices[i][j]
        nearest_species.update(data_sans_doublons.loc[index]['speciesId'])
    result.append(list(nearest_species))

In [85]:
# Données utilisées par l'algo 
X1 = blind_abiotique[['bio1', 'bio2','bio3','bio4','bio5','bio6','bio7','bio8','bio9','bio1','bio10',
                      'bio11','bio12','bio13','bio14','bio15','bio16','bio17','bio18','bio19']].values
X2 = data_sans_doublons[['lat', 'lon']].values
    
    # Création du modèle
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(X2)
    # le modèle retourne l'indice des k-voisins ainsi que leurs distances du point
    distances, indices = nn.kneighbors(X1)
    
    # Récupération des espèces selon les indices des coordonnées des voisins
    result = []
    for i in range(len(test)):
        nearest_species = set()
        for j in range(len(indices[i])):
            index = indices[i][j]
            nearest_species.update(data_sans_doublons.loc[index]['speciesId'])
        result.append(list(nearest_species))
    
    #traitement pour l'export du CSV au format acceptable par Kaggle
    test['nearest_species'] = result
    test = test[['Id', 'nearest_species']]
    test = test.rename(columns={'nearest_species': 'Predicted'})    
    test['Predicted'] = test['Predicted'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace(',', ''))
    test.to_csv('30-voisins.csv', index=False, sep=',')

    return test

IndentationError: unexpected indent (1445169950.py, line 7)

In [43]:
data_sans_doublons = presences.groupby(['patchID'])['speciesId'].apply(list).reset_index()
data_sans_doublons = data_sans_doublons.merge(donnees_abiotiques, on='patchID', how='inner')
data_sans_doublons.to_csv('pa_listEspeces_donneesAbiotiques.csv')